# Final Project: License Plate Reader


Manual license plate reading is prone to errors, especially when there are multiple vehicles, or the plates are dirty, obstructed or damaged. Automated license plate reading (ALPR) using computer vision can improve accuracy, speed up the process, and reduce human error. By using methodologies that we used in class such as image processing/transformation applications and feature matching, we should be able to have our code correctly identify the edges and boundaries of each character to successfully read a license plate.

In order to achieve this, we will be going through the following process:
- Be able to identify the placement of the license plate.
- Identify each character on the license plate.
- Classify each character and add up to get the full number.